In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import keras
import numpy as np #библиотека для работы с массивами данных
from tensorflow.keras.models import Sequential
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Flatten, Dense, Conv2D, Dropout, Input, Conv3D, MaxPooling2D, Conv1D, MaxPooling1D, BatchNormalization
from keras.utils import to_categorical
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from PIL import Image
from tensorflow.keras.preprocessing import image
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
letter_x=[]
for i in os.listdir('all_letters_image'):
    file = keras.utils.load_img('all_letters_image/{i}'.format(i=i),target_size = (28,28),color_mode='grayscale')
    file = image.img_to_array(file)
    letter_x.append(file)

In [11]:
letter_x = np.array(letter_x)
letter_x.shape
print(letter_x[1])

In [12]:
letters_y=[]
for y in os.listdir('all_letters_image'):
    if y[:1]=='0':
        letters_y.append(y[1:2])
    elif y[:1]!='0':
        letters_y.append(y[:2])

In [13]:
y_train_cat = to_categorical(letters_y, num_classes=34)

print(y_train_cat[1])
y_train_cat.shape

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


(14193, 34)

In [14]:
x_train,x_test,y_train,y_test = train_test_split(letter_x,y_train_cat,train_size=0.725, shuffle=True)

In [15]:
print(x_test.shape,x_train.shape,y_train.shape,y_test.shape)

(3904, 28, 28, 1) (10289, 28, 28, 1) (10289, 34) (3904, 34)


In [16]:
#x_train = x_train.reshape(10289,784)
#x_test = x_test.reshape(3904,784)
#print(x_train.shape,x_test.shape)

In [17]:
#print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

In [18]:
x_train = x_train.astype('float')
x_train = x_train/255.
x_test = x_test.astype('float')
x_test = x_test/255.

In [53]:
inp = Input(shape=(28, 28, 1))
# x = Conv2D(128, (9,9), padding='same', name='conv2_1', activation='relu')(inp)
# x = MaxPooling2D((2, 2), strides=2)(x)
x = Conv2D(32, (3,3), padding='same', name='conv2_1', kernel_initializer='he_uniform', activation='relu')(inp)
x = BatchNormalization()(x)
x = Conv2D(32, (3,3), padding='same', name='conv2_2', kernel_initializer='he_uniform', activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.2)(x)
x = Conv2D(64, (3,3), padding='same', name='conv2_3', kernel_initializer='he_uniform', activation='relu')(inp)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), padding='same', name='conv2_4', kernel_initializer='he_uniform', activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.3)(x)
x = Flatten(name='flatten')(x)
out = Dense(34, activation='softmax', name='dense_out')(x)
model = Model(inp, out)

In [54]:
optimizer = keras.optimizers.Adam(learning_rate = 0.0001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_3 (Conv2D)                │ (None, 28, 28, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_40          │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_4 (Conv2D)                │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_out (Dense)               │ (None, 34)             │       426,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 464,610 (1.77 MB)

 Trainable params: 464,354 (1.77 MB)

 Non-trainable params: 256 (1.00 KB)

None


In [ ]:
callbacks = [ModelCheckpoint('./cls/model.weights.h5', monitor='val_accuracy', mode='max', save_weights_only=True, save_best_only=True),
             ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, mode='auto', min_delta=0.001, min_lr=0.000001)]

history = model.fit(x_train, y_train, validation_data=(x_test, y_test),batch_size=128, epochs=40, verbose=1,callbacks=callbacks)

plt.plot(history.history['accuracy'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()
plt.plot(history.history['loss'], 
         label='Доля ошибок на обучающем наборе')
plt.plot(history.history['val_loss'], 
         label='Доля ошибок на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
prediction = model.predict(x_test)

In [ ]:
classes = ['none','а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']

n = 1241

#Выводим на экран результаты
print("Выход сети: ", prediction[n])
print("Распознанный образ: ", np.argmax(prediction[n]))
print("Верный ответ: ", y_test[n])
print("Распознанный браз на картинке: ", classes[np.argmax(prediction[n])])

In [66]:
classes = ['none','а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']

n = 1241

#Выводим на экран результаты
print("Выход сети: ", prediction[n])
print("Распознанный образ: ", np.argmax(prediction[n]))
print("Верный ответ: ", y_test[n])
print("Распознанный браз на картинке: ", classes[np.argmax(prediction[n])])

Выход сети:  [1.4666517e-08 6.3375919e-03 3.0379690e-07 1.6308335e-05 2.6277683e-04
 1.0756483e-04 1.6871306e-04 4.2634259e-08 4.5529965e-04 2.0420296e-06
 2.1641856e-04 3.3144828e-05 3.4878820e-03 1.5171112e-04 1.3357113e-04
 2.1048638e-03 8.0279488e-04 9.6093702e-01 5.8873447e-06 1.3841998e-05
 2.1117628e-02 1.1763612e-04 3.6998588e-04 6.0018367e-04 1.0841380e-05
 5.5842206e-04 1.8460214e-05 1.2644776e-04 1.5425454e-05 1.9760424e-04
 5.2306405e-07 3.2615578e-08 1.6101731e-03 1.8737655e-05]
Распознанный образ:  17
Верный ответ:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Распознанный браз на картинке:  п
